# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [180]:
from dataclasses import dataclass
import math
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.value * o_d + (-self.d * o_value)) * o_value ** (-2))
            case Number():
                return Dual(self.value / other, self.d / other)                 
     
    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value**o_value, self.value**o_value*math.log(self.value))
            case Number():
                return Dual(self.value ** float(other), other*self.value**float(other-1)*self.d)      

    def __neg__(self) -> "Dual":
         return Dual(-self.value, -self.d)    

    def __pos__(self) -> "Dual":
         return Dual(+self.value, +self.d)      

    def __abs__(self) -> "Dual":
         if (self.value >= 0):
            return Dual(self.value, self.d) 
         else:
            return Dual(-1*self.value, -1*self.d)          

    def __invert__(self) -> "Dual":
         return Dual(-1*(self.value+1), -1*(self.d))     
    
    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    # __lpow__ = __pow__

 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d       


8.0

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [181]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return x*x

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(4)

8.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [194]:
# ваш код
# Недостатки: неправильно считалась производная произведения

def f(x: float) -> float:
    # return (x**2)/(x+1) # проверяется производная от частного ==== -0.8888888888888888; здесь и далее x=2
    # return Dual(2, 0)**x # проверяется производная от показательной функции (также степененной) ==== 2.772588722239781
    # return +x+3 # проверяется производная от унарного плюса  ==== 1.0  
    # return -x+3 # проверяется производная от унарного плюса  ==== -1.0
    # return ~x+3 # проверяется производная от унарного inverse  ==== -1.0
    return abs(-x+3) # проверяется производная от функции модуля  ==== -1.0  

f_diff = diff(f)
f_diff(2)


-1.0

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [197]:
# ваш код

import math

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

e = Dual(math.exp(1), 0)
def exp(arg: Dual) -> "Dual":
    return Dual(e.value**arg.value, (e.value**arg.value)*arg.d) # реализация функции exp - по сути обычная показательная

def cos(arg: Dual) -> "Dual":
    return Dual(math.cos(arg.value), -math.sin(arg.value)*arg.d) #реализация функции косинуса для производной

def sin(arg: Dual) -> "Dual":
    return Dual(math.sin(arg.value), math.cos(arg.value)*arg.d) #реализация функции синуса для производной

def log(arg: Dual, base: float) -> "Dual":
    return Dual(math.log(arg.value, base), 1 / (arg.value * math.log(base) * arg.d)) #реализация функции логарифма с произвольным основанием для производной




def f(x: float) -> float:
    # return exp(2*x) # значение для x=1
    # return cos(3*x) # значения при X=PI/2 ==== 2.999991439163154
    # return sin(3*x)  # значения при X=PI/2 ==== -0.007166934336844616
    return log(x, e.value) # значение при x = 5, a = e (2.71) =====  0.2

f_diff = diff(f)
f_diff(1)


14.778112197861299

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [199]:
from scipy.misc import derivative
import math

def f(x: float) -> float:
    # return (x**2)/(x+1) # проверяется производная от частного ==== -0.8888888888888888; здесь и далее x=2 +
    # return 2**x # проверяется производная от показательной функции (также степененной) ==== 2.772588722239781 +
    # return +x+3 # проверяется производная от унарного плюса  ==== 1.0   +
    # return -x+3 # проверяется производная от унарного плюса  ==== -1.0 +
    # return ~int(x)+3 # проверяется производная от унарного inverse  ==== -1.0 + 
    # return abs(-x+3) # проверяется производная от функции модуля  ==== -1.0  +
    # return math.exp(x) # значение для x=1 (почему то e^x производная при x = 1 в этой функции равно 3.19, хотя должно быть 2.71) -
    # return math.cos(3*x) # значения при X=PI/2 ==== 2.999991439163154 (ошибка также: производная cos(3*x) при x = 3*PI/2 = -3*sin(3PI/2) = 1, а библиотека выдает 0.14) - 
    # return math.sin(3*x)  # значения при X=PI/2 ==== -0.007166934336844616 +
    return math.log(x, 2.71) # значение при x = 5, a = e (2.71) =====  0.2 +

derivative(f, 5)

0.20335305848141683

In [ ]:
# ваш код

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [118]:
# ваш код
import ast
import math
import string
from typing import List
import random
from typing import Union, Callable
from numbers import Number
from dataclasses import dataclass

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.value * o_d + (-self.d * o_value)) * o_value ** (-2))
            case Number():
                return Dual(self.value / other, self.d / other)                 
     
    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value**o_value, self.value**o_value*math.log(self.value))
            case Number():
                return Dual(self.value ** float(other), other*self.value**float(other-1)*self.d)      

    def __neg__(self) -> "Dual":
         return Dual(-self.value, -self.d)    

    def __pos__(self) -> "Dual":
         return Dual(+self.value, +self.d)      

    def __abs__(self) -> "Dual":
         if (self.value >= 0):
            return Dual(self.value, self.d) 
         else:
            return Dual(-1*self.value, -1*self.d)          

    def __invert__(self) -> "Dual":
         return Dual(-1*(self.value+1), -1*(self.d))     
    
    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

def cos(arg: Dual) -> "Dual":
    return Dual(math.cos(arg.value), -math.sin(arg.value)*arg.d) #реализация функции косинуса для производной

def sin(arg: Dual) -> "Dual":
    return Dual(math.sin(arg.value), math.cos(arg.value)*arg.d) #реализация функции синуса для производной

def log(arg: Dual, base: float) -> "Dual":
    return Dual(math.log(arg.value, base), 1 / (arg.value * math.log(base) * arg.d))

arr_symbol_dif: List = []
arr_num_dif: List = []

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#

NUM_OF_FUNCTIONS = 10 #можете задать любое число
EXPECTED_X_VALUE = 2 #можете задать точку, в которой будет считаться производная

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#

for i in range(NUM_OF_FUNCTIONS):
     rand = random.randrange(5)
     temprand = random.randrange(3, 9)
     match rand:
          case 0:
               arr_symbol_dif.append('lambda x: sin({}*x)'.format(temprand))
               arr_num_dif.append('lambda x: math.sin({}*x)'.format(temprand))
          case 1:
               arr_symbol_dif.append('lambda x: cos({}*{}*x)'.format(-1, temprand - rand))
               arr_num_dif.append('lambda x: math.cos({}*{}*x)'.format(-1, temprand - rand))
          case 2:
               arr_symbol_dif.append('lambda x: {}*x'.format((-1)**temprand))
               arr_num_dif.append('lambda x: {}*x'.format((-1)**temprand))
          case 3:
               arr_symbol_dif.append('lambda x: +x')
               arr_num_dif.append('lambda x: +x')
          case 4:
               arr_symbol_dif.append('lambda x: {}*x**{}'.format(i, temprand))  
               arr_num_dif.append('lambda x: {}*x**{}'.format(i, temprand))               

for i in range(NUM_OF_FUNCTIONS):
     rand = random.randrange(1,3)
     temprand = random.randrange(3, 9)
     if rand % 2 == 0:
          arr_symbol_dif[i] += ' + (-1)*{}**x'.format(Dual(temprand - rand, 0))
          arr_num_dif[i] += ' + (-1)*{}**x'.format(temprand - rand)
     if temprand % 2 == 0:
          arr_symbol_dif[i] += ' + log(x, {})*(-{})'.format(temprand, temprand) 
          arr_num_dif[i] += ' + math.log(x, {})*(-{})'.format(temprand, temprand) 
     if temprand % 3 == 0:
          arr_symbol_dif[i] += ' + sin(log({}*x**2 + 3*x, {}))+cos(log(({})**x, 2))'.format(rand/temprand, temprand, Dual(temprand + 3, 0)) 
          arr_num_dif[i] += ' + math.sin(math.log({}*x**2 + 3*x, {}))+ math.cos(math.log(({})**x, 2))'.format(rand/temprand, temprand, temprand + 3) 

func_symdif_arr = [eval(i) for i in arr_symbol_dif]
func_numdif_arr = [eval(i) for i in arr_num_dif]

sym_diff_func_arr = [diff(func) for func in func_symdif_arr] 

# print()
for i in range(len(arr_symbol_dif)):
    x = EXPECTED_X_VALUE # random.randrange(2, 4)
    print('=====================================')
    print("Results of both (SYMBOLIC: {}) and (NUMERICAL: {}) differentiation on FUNCTION({}): {}".format(sym_diff_func_arr[i](x), derivative(func_numdif_arr[i], x), x, arr_num_dif[i][10:]))

##Если Вы проверите результаты, вы увидите, что некоторые отличаются. Причина - derivative дифференцирует плохо, например 2^x в точке 2 для него =3, хотя на самом деле 2.77
# derivative(eval('lambda x: 2**x'), 2)





Results of both (SYMBOLIC: -2.8581372120810586) and (NUMERICAL: -1.160602264655227) differentiation on FUNCTION(4): math.cos(-1*7*x) + math.log(x, 8)*(-8)
Results of both (SYMBOLIC: 0.038203306074024335) and (NUMERICAL: 0.017379207778392303) differentiation on FUNCTION(4): +x + math.log(x, 8)*(-8)
Results of both (SYMBOLIC: 2.0358081904001035) and (NUMERICAL: -0.28740616909235955) differentiation on FUNCTION(4): math.sin(5*x) + (-1)*1**x + math.sin(math.log(0.6666666666666666*x**2 + 3*x, 3))+ math.cos(math.log((6)**x, 2))
Results of both (SYMBOLIC: -15.642340330697145) and (NUMERICAL: -12.012206509802176) differentiation on FUNCTION(4): math.sin(4*x) + (-1)*2**x + math.log(x, 4)*(-4)
Results of both (SYMBOLIC: 2.0358081904001035) and (NUMERICAL: -0.28740616909235955) differentiation on FUNCTION(4): math.sin(5*x) + (-1)*1**x + math.sin(math.log(0.6666666666666666*x**2 + 3*x, 3))+ math.cos(math.log((6)**x, 2))
Results of both (SYMBOLIC: 1.0) and (NUMERICAL: 1.0) differentiation on FUNCTI

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [193]:
# ваш код

@dataclass
class Ternary:
    value: float
    gradient: tuple[str, float]

    def __add__(self, other: Union["Ternary", Number]) -> "Ternary":
        match other:
            case Ternary(o_value, o_gradient):
                return Ternary(self.value + o_value, (self.gradient[0] + o_gradient[0], self.gradient[1] + o_gradient[1], self.gradient[2] + o_gradient[2]))
            case Number():
                return Ternary(float(other) + self.value, (self.gradient[0], self.gradient[1], self.gradient[2]))

    def __mul__(self, other: Union["Ternary", Number]) -> "Ternary":
         match other:
            case Ternary(o_value, o_gradient):
                return Ternary(self.value + o_value, (self.gradient[0] * o_value + self.value * o_gradient[0], 
                                                      self.gradient[1] * o_value + self.value * o_gradient[1], 
                                                      self.gradient[2] * o_value + self.value * o_gradient[2]))
            case Number():
                return Ternary(float(other) * self.value, (self.gradient[0] * float(other), self.gradient[1] * float(other), self.gradient[2] * float(other)))

    def __neg__(self) -> "Ternary":
         return Ternary(-self.value, (-self.gradient[0], -self.gradient[1], -self.gradient[2]))    
    
    def __neg__(self) -> "Ternary":
         return Ternary(+self.value, (+self.gradient[0], +self.gradient[1], +self.gradient[2])) 

    def __invert__(self) -> "Ternary":
         return Ternary(-1*(self.value+1), (-self.gradient[0], -self.gradient[1], -self.gradient[2]))  

    __radd__ = __add__  
    __rmul__ = __mul__      
            
    

def f(x: float, y: float, z: float) -> float:
    return x * y + z + (-5) * y  


def extra_diff(func: Callable[[float, float, float], List[float]]) -> Callable[[float, float, float], List[float]]:
    return lambda x, y, z: func(Ternary(x, (1, 0, 0)), Ternary(y, (0, 1, 0)), Ternary(z, (0, 0, 1))).gradient

f_diff = extra_diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]

(10.0, 5.0, 1.0)